In [ ]:
# 1. Εγκατάσταση εξαρτήσεων συστήματος (Linux libs)
!apt-get update
!apt-get install -y ffmpeg libavcodec-dev libavformat-dev libavdevice-dev libavfilter-dev libavutil-dev libswscale-dev libswresample-dev

# 2. Τώρα δοκίμασε ξανά την εγκατάσταση του Audiocraft
!pip install -U git+https://git@github.com/facebookresearch/audiocraft#egg=audiocraft
!pip install torchaudio

In [ ]:
# 1. Καθαρισμός conflicting libraries
!pip uninstall -y tensorflow tensorboard protobuf

# 2. Εγκατάσταση 'φρέσκου' περιβάλλοντος για Audio
!apt-get update
!apt-get install -y ffmpeg libavcodec-dev libavformat-dev libavdevice-dev libavfilter-dev libavutil-dev libswscale-dev libswresample-dev

# 3. Εγκατάσταση Audiocraft με συγκεκριμένη σειρά
!pip install "protobuf<3.20"  # Downgrade protobuf για να μην χτυπάει
!pip install torchaudio
!pip install -U git+https://git@github.com/facebookresearch/audiocraft#egg=audiocraft

In [ ]:
# 1. Απεγκατάσταση της προβληματικής βιβλιοθήκης
!pip uninstall -y torchao

# 2. Εγκατάσταση απαραίτητων εργαλείων συστήματος (για να μην σκάσει το av)
!apt-get update
!apt-get install -y ffmpeg libavcodec-dev libavformat-dev libavdevice-dev libavfilter-dev libavutil-dev libswscale-dev libswresample-dev

# 3. Εγκατάσταση MusicGen (Audiocraft)
# Βάζουμε --no-deps στο audiocraft για να μην μας αλλάξει εκδόσεις torch που δουλεύουν
!pip install -U git+https://git@github.com/facebookresearch/audiocraft#egg=audiocraft

# 4. Εγκατάσταση βιβλιοθηκών που μπορεί να λείπουν, σε σταθερές εκδόσεις
!pip install librosa torchaudio

In [3]:
import os

print("🔍 Ψάχνω για αρχεία ήχου στο /kaggle/input...")

found_files = []
# Ψάχνουμε αναδρομικά σε όλους τους φακέλους
for root, dirs, files in os.walk("/kaggle/input"):
    for file in files:
        if file.endswith((".wav", ".mp3")):
            full_path = os.path.join(root, file)
            found_files.append(full_path)
            # Τυπώνουμε τα 3 πρώτα για να δούμε το Path
            if len(found_files) <= 3:
                print(f"✅ Βρέθηκε: {full_path}")

if not found_files:
    print("❌ Δεν βρέθηκε κανένα αρχείο ήχου! Βεβαιώσου ότι ανέβασες το Dataset.")
else:
    print(f"\nΣυνολικά βρέθηκαν {len(found_files)} αρχεία.")
    # Αυτόματη εύρεση του σωστού φακέλου
    suggested_dir = os.path.dirname(found_files[0])
    print(f"\n💡 Ο σωστός φάκελος (AUDIO_DIR) μάλλον είναι:\n'{suggested_dir}'")

🔍 Ψάχνω για αρχεία ήχου στο /kaggle/input...
✅ Βρέθηκε: /kaggle/input/thesis-audio-files/pair_41/original.wav
✅ Βρέθηκε: /kaggle/input/thesis-audio-files/pair_41/suspicious.wav
✅ Βρέθηκε: /kaggle/input/thesis-audio-files/pair_17/original.wav

Συνολικά βρέθηκαν 100 αρχεία.

💡 Ο σωστός φάκελος (AUDIO_DIR) μάλλον είναι:
'/kaggle/input/thesis-audio-files/pair_41'


In [5]:
import torchaudio
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write
import os
import random
from tqdm import tqdm
import glob

# --- ΡΥΘΜΙΣΕΙΣ ---
# Ο κεντρικός φάκελος (θα ψάξουμε μέσα σε αυτόν παντού)
BASE_INPUT_DIR = "/kaggle/input/thesis-audio-files" 

# Πού να σώσω τα covers;
OUTPUT_DIR = "/kaggle/working/generated_covers"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Πόσα covers θες να φτιάξεις;
NUM_COVERS = 30  # Βάλε π.χ. 30 ή 50
DURATION = 30    # Δευτερόλεπτα

# Στυλ (Hard Positives)
STYLES = [
    "heavy metal rock song, energetic, distorted guitar",
    "smooth jazz piano, relaxing, lounge atmosphere",
    "electronic dance music, techno, synth bass, 140 bpm",
    "classical orchestral symphony, violins, dramatic",
    "reggae song, slow beat, jamaican style",
    "acoustic guitar folk song, campfire vibe",
    "8-bit chiptune video game music, retro",
    "hip hop beat, old school, boom bap drums"
]

# --- 1. ΕΥΡΕΣΗ ΑΡΧΕΙΩΝ (ΤΟ FIX) ---
print("🔍 Scanning for original.wav files...")
# Ψάχνουμε παντού για αρχεία που λέγονται 'original.wav'
# Το ** σημαίνει "σε όλο το βάθος των φακέλων"
all_originals = glob.glob(os.path.join(BASE_INPUT_DIR, "**", "original.wav"), recursive=True)

# Αν δεν βρει original.wav, ψάχνουμε για οποιοδήποτε wav
if not all_originals:
    all_originals = glob.glob(os.path.join(BASE_INPUT_DIR, "**", "*.wav"), recursive=True)

print(f"✅ Found {len(all_originals)} source files.")

if len(all_originals) == 0:
    raise ValueError("Δεν βρέθηκαν αρχεία! Έλεγξε το path.")

# Επιλογή τυχαίων τραγουδιών αν έχουμε πολλά
if len(all_originals) > NUM_COVERS:
    selected_files = random.sample(all_originals, NUM_COVERS)
else:
    selected_files = all_originals

# --- 2. ΦΟΡΤΩΣΗ ΜΟΝΤΕΛΟΥ ---
print("⏳ Loading MusicGen...")
model = MusicGen.get_pretrained('facebook/musicgen-melody')
model.set_generation_params(duration=DURATION)

# --- 3. GEN LOOP ---
print(f"🚀 Generating {len(selected_files)} AI Covers...")

for idx, file_path in enumerate(tqdm(selected_files)):
    try:
        # Φόρτωση Original (Melody)
        melody, sr = torchaudio.load(file_path)
        
        # Αν το τραγούδι είναι μικρότερο από 30 δευτ, το αγνοούμε ή κόβουμε
        if melody.shape[1] / sr < 5.0: # Skip very short files
            continue

        # Επιλογή Τυχαίου Στυλ
        style = random.choice(STYLES)
        description = [style]
        
        # Generation
        wav = model.generate_with_chroma(
            descriptions=description,
            melody_wavs=melody[None], 
            melody_sample_rate=sr
        )
        
        # Save
        wav = wav.detach().cpu().squeeze(0)
        # Φτιάχνουμε όνομα αρχείου που να θυμίζει από πού ήρθε
        # π.χ. cover_pair41_jazz
        parent_folder = os.path.basename(os.path.dirname(file_path)) # pair_41
        safe_style = style.split(',')[0].replace(' ', '_')
        save_name = f"cover_{parent_folder}_{safe_style}_{idx}"
        
        save_path = os.path.join(OUTPUT_DIR, save_name)
        audio_write(save_path, wav, model.sample_rate, strategy="loudness", loudness_headroom_db=16)
        
    except Exception as e:
        print(f"❌ Error on {file_path}: {e}")
        continue

print(f"✅ Έτοιμα! Τα covers είναι στο {OUTPUT_DIR}")

🔍 Scanning for original.wav files...
✅ Found 51 source files.
⏳ Loading MusicGen...


state_dict.bin:   0%|          | 0.00/2.77G [00:00<?, ?B/s]

Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/955717e8-8726e21a.th" to /root/.cache/torch/hub/checkpoints/955717e8-8726e21a.th
100%|██████████| 80.2M/80.2M [00:00<00:00, 143MB/s]


compression_state_dict.bin:   0%|          | 0.00/236M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


🚀 Generating 30 AI Covers...


100%|██████████| 30/30 [50:41<00:00, 101.39s/it]

✅ Έτοιμα! Τα covers είναι στο /kaggle/working/generated_covers


In [6]:
import shutil
import os

# Ο φάκελος που περιέχει τα covers που μόλις έφτιαξες
SOURCE_DIR = "/kaggle/working/generated_covers"
# Το όνομα του zip αρχείου που θα δημιουργηθεί
OUTPUT_FILENAME = "/kaggle/working/ai_covers_audio"

print(f"📦 Zipping files from {SOURCE_DIR}...")

if os.path.exists(SOURCE_DIR) and len(os.listdir(SOURCE_DIR)) > 0:
    # Δημιουργία του ZIP
    shutil.make_archive(OUTPUT_FILENAME, 'zip', SOURCE_DIR)
    
    print(f"\n✅ ΕΤΟΙΜΟ! Το αρχείο δημιουργήθηκε: {OUTPUT_FILENAME}.zip")
    print("👉 Πήγαινε δεξιά στο μενού 'Output', κάνε refresh αν χρειαστεί, και κατέβασε το 'ai_covers_audio.zip'.")
else:
    print("❌ Σφάλμα: Ο φάκελος είναι άδειος ή δεν υπάρχει.")

📦 Zipping files from /kaggle/working/generated_covers...

✅ ΕΤΟΙΜΟ! Το αρχείο δημιουργήθηκε: /kaggle/working/ai_covers_audio.zip
👉 Πήγαινε δεξιά στο μενού 'Output', κάνε refresh αν χρειαστεί, και κατέβασε το 'ai_covers_audio.zip'.


In [8]:
from IPython.display import FileLink

# Βεβαιώσου ότι το όνομα είναι σωστό (χωρίς το /kaggle/working/ μπροστά, μόνο το όνομα αρχείου)
# Αν το zip σου είναι στο /kaggle/working/ai_covers_audio.zip
FileLink('ai_covers_audio.zip')

/kaggle/working/ai_covers_audio.zip

In [ ]:
# Εγκατάσταση MERT dependencies (αν δεν υπάρχουν)
!pip install transformers torchaudio

In [ ]:
# Εγκατάσταση μιας σταθερής έκδοσης transformers που δεν έχει το "security panic"
!pip install transformers==4.46.3

# Επιβεβαίωση ότι έχουμε τα βασικά
!pip install torchaudio librosa numpy

In [10]:
import shutil
import os

# Ο φάκελος που περιέχει τα covers που μόλις έφτιαξες
SOURCE_DIR = "/kaggle/working/ai_covers_segmented"
# Το όνομα του zip αρχείου που θα δημιουργηθεί
OUTPUT_FILENAME = "/kaggle/working/ai_covers_audio_segments"

print(f"📦 Zipping files from {SOURCE_DIR}...")

if os.path.exists(SOURCE_DIR) and len(os.listdir(SOURCE_DIR)) > 0:
    # Δημιουργία του ZIP
    shutil.make_archive(OUTPUT_FILENAME, 'zip', SOURCE_DIR)
    
    print(f"\n✅ ΕΤΟΙΜΟ! Το αρχείο δημιουργήθηκε: {OUTPUT_FILENAME}.zip")
    print("👉 Πήγαινε δεξιά στο μενού 'Output', κάνε refresh αν χρειαστεί, και κατέβασε το 'ai_covers_audio_segments.zip'.")
else:
    print("❌ Σφάλμα: Ο φάκελος είναι άδειος ή δεν υπάρχει.")

from IPython.display import FileLink

# Βεβαιώσου ότι το όνομα είναι σωστό (χωρίς το /kaggle/working/ μπροστά, μόνο το όνομα αρχείου)
# Αν το zip σου είναι στο /kaggle/working/ai_covers_audio.zip
FileLink('ai_covers_audio_segments.zip')

📦 Zipping files from /kaggle/working/ai_covers_segmented...

✅ ΕΤΟΙΜΟ! Το αρχείο δημιουργήθηκε: /kaggle/working/ai_covers_audio_segments.zip
👉 Πήγαινε δεξιά στο μενού 'Output', κάνε refresh αν χρειαστεί, και κατέβασε το 'ai_covers_audio_segments.zip'.


/kaggle/working/ai_covers_audio_segments.zip

In [9]:
import os
import librosa
import soundfile as sf
import numpy as np
from tqdm import tqdm

# --- ΡΥΘΜΙΣΕΙΣ ---
INPUT_DIR = "/kaggle/working/generated_covers"      # Εδώ είναι τα wav που έφτιαξε το MusicGen
OUTPUT_DIR = "/kaggle/working/ai_covers_segmented"  # Εδώ θα μπουν τα κομμάτια
BEATS_PER_SEGMENT = 16  # Η ρύθμιση από το segment_audio.py σου

# Εγκατάσταση librosa αν λείπει
try:
    import librosa
except ImportError:
    os.system('pip install librosa soundfile')
    import librosa

def segment_track_beat_synced(file_path, output_folder):
    """
    Διαβάζει ένα wav, βρίσκει τα beats και το κόβει ανά 16 beats.
    Βασισμένο στο segment_audio.py σου.
    """
    try:
        # Φόρτωση
        y, sr = librosa.load(file_path, sr=None)
        
        # Beat Tracking
        tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
        beat_samples = librosa.frames_to_samples(beat_frames)
        
        # Αν δεν βρει αρκετά beats (π.χ. πολύ μικρό ή ambient), το αγνοούμε
        if len(beat_samples) < BEATS_PER_SEGMENT:
            print(f"⚠️ Warning: Λίγα beats στο {os.path.basename(file_path)}. Skipping.")
            return

        # Segmentation Loop
        num_segments = 0
        for i in range(0, len(beat_samples) - BEATS_PER_SEGMENT, BEATS_PER_SEGMENT):
            start_sample = beat_samples[i]
            end_sample = beat_samples[i + BEATS_PER_SEGMENT]
            
            segment = y[start_sample:end_sample]
            
            # Αγνοούμε πολύ μικρά segments (< 2 sec) για να μην χαλάσουν το MERT
            if len(segment) / sr < 2.0:
                continue

            # Αποθήκευση: 0.wav, 1.wav, κλπ.
            seg_filename = f"{num_segments}.wav"
            sf.write(os.path.join(output_folder, seg_filename), segment, sr)
            num_segments += 1
            
    except Exception as e:
        print(f"❌ Error processing {file_path}: {e}")

# --- MAIN EXECUTION ---
print("🚀 Starting 4-Bar Segmentation on AI Covers...")

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Βρίσκουμε τα wav files
cover_files = [f for f in os.listdir(INPUT_DIR) if f.endswith('.wav')]
print(f"📂 Found {len(cover_files)} covers to process.")

for filename in tqdm(cover_files):
    input_path = os.path.join(INPUT_DIR, filename)
    
    # Δημιουργία υποφακέλου για κάθε τραγούδι
    # Π.χ. ai_covers_segmented/cover_pair_41_jazz_0/
    song_name = os.path.splitext(filename)[0]
    song_output_dir = os.path.join(OUTPUT_DIR, song_name)
    os.makedirs(song_output_dir, exist_ok=True)
    
    # Εκτέλεση Segmentation
    segment_track_beat_synced(input_path, song_output_dir)

print(f"\n✅ Segmentation Complete! Files are in {OUTPUT_DIR}")

🚀 Starting 4-Bar Segmentation on AI Covers...
📂 Found 30 covers to process.


100%|██████████| 30/30 [00:17<00:00,  1.70it/s]


✅ Segmentation Complete! Files are in /kaggle/working/ai_covers_segmented


In [14]:
import torch
import os
import numpy as np
from transformers import AutoModel, Wav2Vec2FeatureExtractor
import torchaudio
from tqdm import tqdm
import glob

# --- ΡΥΘΜΙΣΕΙΣ PATHS ---
# Ο φάκελος που περιέχει τους ΥΠΟΦΑΚΕΛΟΥΣ με τα segments
# Π.χ. /kaggle/working/ai_covers_segmented/cover_pair_41_jazz_0/
INPUT_SEGMENTED_DIR = "/kaggle/working/ai_covers_segmented"

# Ο φάκελος που θα σωθούν τα τελικά .npy (ένα αρχείο ανά cover)
OUTPUT_EMB_DIR = "/kaggle/working/hard_positive_embeddings"
os.makedirs(OUTPUT_EMB_DIR, exist_ok=True)

# --- 1. Η ΣΥΝΑΡΤΗΣΗ ΣΟΥ (ΑΜΕΤΑΒΛΗΤΗ) ---
def run_mert_model_and_get_features(waveforms, audio_model, time_reduce=None):
    if time_reduce is None:
        time_reduce = torch.nn.AvgPool1d(kernel_size=10, stride=10, count_include_pad=False)
    
    outputs = audio_model(waveforms, output_hidden_states=True)
    hidden_states = outputs.hidden_states
    
    # Selection of specific layers [2, 5, 8, 11] -> 4 Layers total
    audio_features = torch.stack(
        [time_reduce(h.detach()[:, :, :].permute(0,2,1)).permute(0,2,1) for h in hidden_states[2::3]], dim=1
    )
    return audio_features


def extract_hard_positives_structured():
    print("🚀 Starting Structured MERT Extraction (Robust Save)...")
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    processor = Wav2Vec2FeatureExtractor.from_pretrained("m-a-p/MERT-v1-95M")
    model = AutoModel.from_pretrained("m-a-p/MERT-v1-95M", trust_remote_code=True).to(device)
    model.eval()
    
    time_reduce = torch.nn.AvgPool1d(kernel_size=10, stride=10, count_include_pad=False).to(device)

    # Βρίσκουμε τους φακέλους
    song_folders = [f for f in os.listdir(INPUT_SEGMENTED_DIR) if os.path.isdir(os.path.join(INPUT_SEGMENTED_DIR, f))]
    print(f"📂 Found {len(song_folders)} cover songs (folders).")
    
    for song_folder in tqdm(song_folders):
        song_path = os.path.join(INPUT_SEGMENTED_DIR, song_folder)
        
        # Ταξινόμηση
        segment_files = sorted(
            [f for f in os.listdir(song_path) if f.endswith('.wav')],
            key=lambda x: int(os.path.splitext(x)[0])
        )
        
        if not segment_files:
            continue
            
        all_segments_features = []
        
        for seg_file in segment_files:
            seg_path = os.path.join(song_path, seg_file)
            
            # Φόρτωση
            waveform, sr = torchaudio.load(seg_path)
            if sr != 24000:
                resampler = torchaudio.transforms.Resample(sr, 24000)
                waveform = resampler(waveform)
            
            # Extraction
            inputs = processor(waveform.squeeze(), sampling_rate=24000, return_tensors="pt", padding=True)
            input_values = inputs.input_values.to(device)
            
            with torch.no_grad():
                features = run_mert_model_and_get_features(input_values, model, time_reduce)
            
            # (4, Time, 768)
            all_segments_features.append(features.cpu().squeeze(0).numpy())
            
        # --- FIX: Χειροκίνητη δημιουργία Object Array ---
        # Φτιάχνουμε έναν άδειο πίνακα objects με το σωστό μήκος
        # Αυτό αποτρέπει το NumPy από το να προσπαθήσει να κάνει broadcast
        ragged_array = np.empty(len(all_segments_features), dtype=object)
        
        for i, feat in enumerate(all_segments_features):
            ragged_array[i] = feat
            
        save_name = f"{song_folder}.npy"
        np.save(os.path.join(OUTPUT_EMB_DIR, save_name), ragged_array)

    print(f"✅ Extraction Complete! Embeddings saved in {OUTPUT_EMB_DIR}")

# Εκτέλεση
extract_hard_positives_structured()

🚀 Starting Structured MERT Extraction (Robust Save)...
Using device: cuda
📂 Found 30 cover songs (folders).


100%|██████████| 30/30 [00:04<00:00,  6.03it/s]

✅ Extraction Complete! Embeddings saved in /kaggle/working/hard_positive_embeddings


In [15]:
import shutil
shutil.make_archive("hard_positives", 'zip', "/kaggle/working/hard_positive_embeddings")

'/kaggle/working/hard_positives.zip'

In [21]:
FileLink('hard_positives.zip')

/kaggle/working/hard_positives.zip